In [5]:
#Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Dataframe with pandas
df = pd.read_csv('../Python/Data/One Patient.CSV')

In [ ]:
df.head()

In [ ]:
df.drop(df.columns[[0,19]], axis = 1 )

In [8]:
#Qualifying Events start at Day 0, easy to find opioid events via MME
after180 = df[df['MME'].notnull() & (df['Days']>=0)]
#Define series to add
days = after180.loc[:,'Days']
add = after180.loc[:,'PAY_DAY_SUPPLY_CNT']
print (days, add)

193      0
194      0
326    706
329    706
Name: Days, dtype: int64 193    5.0
194    5.0
326    5.0
329    5.0
Name: PAY_DAY_SUPPLY_CNT, dtype: float64


In [60]:
#Making list
atrisk = []
#a is a pointer to the first date of 6 month period
a = 0
i = 0
#generalizable loop
while i in range(len(days)):
#i is current event
    currentday= days.iloc[i]
    nextday = days.iloc[i+1]
    fromcurrent180 = days.iloc[i] + 180
    currentsupply = days.iloc[i] + add.iloc[i]
    nextsupply = days.iloc[i+1] + add.iloc[i+1]
    month6begin = days.iloc[a]
    month6end = days.iloc[a] + 180
    begin= str(days.iloc[a])
    end= str(days.iloc[a]+180)
    overlap = 0
    over180 = 0 
    print('6-month window: ' + begin + ' - ' + end)
    print('6month window event:' + str(a))
    print('Currently it is event:' + str(i) + ' Day:' + str(days.iloc[i]) + ' Supply Count: ' + str(add.iloc[i]))
    
#Remove days over 180 days, ignore if not in 6 month window
    if (nextsupply >= fromcurrent180) & (nextday < month6end): 
        over180 = fromcurrent180 - nextsupply
#Remove overlapping days
    print('over180: ' + str(over180))
    if (currentday < month6end) & (currentsupply > days.iloc[i+1]):
        overlap = currentsupply - nextday
    print('overlap: ' + str(overlap))
#90% of 6 month period
    Value = add.iloc[i] - overlap - over180
    print('Value: ' + str(Value))
    if Value >= 162:
        atrisk.append(True)
        break
    else:
        i += 1
        print('Going to event: ' + str(i))
#a goes to next event when 6 month window fails
    if (days.iloc[i] >= month6end) & (Value < 162):
        a += 1
        print('Going to next 6-month window: ' + begin + ' - ' + end)
        print('With event: ' + str(a) + ' as beginning of 6 month window')
        print('It was event: ' + str(i)) 
        i = a
        print("And now we're going back to event: " + str(i))
        Value = 0
    if i+1 >= len(days):
        if (nextsupply >= fromcurrent180) & (nextday < month6end): 
            over180 = fromcurrent180 - nextsupply
            Value += add.iloc[i] - overlap
        if Value < 162:
            atrisk.append(False)
            break
print(atrisk)
    

6-month window: 0 - 180
6month window event:0
Currently it is event:0 Day:0 Supply Count: 5.0
over180: 0
overlap: 5.0
Value: 0.0
Going to event: 1
6-month window: 0 - 180
6month window event:0
Currently it is event:1 Day:0 Supply Count: 5.0
over180: 0
overlap: 0
Value: 5.0
Going to event: 2
Going to next 6-month window: 0 - 180
With event: 1 as beginning of 6 month window
It was event: 2
And now we're going back to event: 1
6-month window: 0 - 180
6month window event:1
Currently it is event:1 Day:0 Supply Count: 5.0
over180: 0
overlap: 0
Value: 5.0
Going to event: 2
Going to next 6-month window: 0 - 180
With event: 2 as beginning of 6 month window
It was event: 2
And now we're going back to event: 2
6-month window: 706 - 886
6month window event:2
Currently it is event:2 Day:706 Supply Count: 5.0
over180: 0
overlap: 5.0
Value: 0.0
Going to event: 3
[False]
